In [ ]:
!nvidia-smi

Mon Dec 21 00:34:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [64]:
!nvidia-smi

Tue Jan 19 21:48:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    30W /  70W |   1647MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Prepare colab environment

In [2]:
!cd /content && export CREDENTIALS=`cat credentials.txt` && git clone https://$CREDENTIALS@github.com/SMAPWr/project-medical_sma

Cloning into 'project-medical_sma'...
remote: Enumerating objects: 400, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (242/242), done.
remote: Total 400 (delta 214), reused 330 (delta 146), pack-reused 0
Receiving objects: 100% (400/400), 14.92 MiB | 24.36 MiB/s, done.
Resolving deltas: 100% (214/214), done.


In [1]:
!cd /content/project-medical_sma/ && git pull

Already up to date.


In [4]:
#!cd /content/project-medical_sma/ && git add dataset/* && git commit -m 'medical_twiteer_embeddings'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys, os
os.chdir('/content/project-medical_sma')
print(os.getcwd())
if os.getcwd() not in sys.path:
  sys.path.insert(0, os.getcwd())
print(sys.path)

/content/project-medical_sma
['/content/project-medical_sma', '', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']


In [57]:
!date
# !cp /content/drive/MyDrive/colab-data/sma-medical/*.npz dataset/
!cp /content/drive/MyDrive/colab-data/sma-medical/dr_herbert/medical_twiteer_embeddings_166.npz dataset/dr_herbert_medical_twitter_embeddings_166.npz
!date
!ls -l dataset/*npz

Tue Jan 19 18:32:50 UTC 2021
Tue Jan 19 18:33:09 UTC 2021
-rw-r--r-- 1 root root    5693825 Jan 19 16:10 dataset/clean_dataset_embeddings.npz
-rw------- 1 root root  806941528 Jan 19 17:10 dataset/dr_herbert_medical_twitter_embeddings_144.npz
-rw------- 1 root root  928719398 Jan 19 18:33 dataset/dr_herbert_medical_twitter_embeddings_166.npz
-rw------- 1 root root  806941528 Jan 19 18:13 dataset/dr_herbert_medical_twitter_embeddings.npz
-rw------- 1 root root  928880121 Jan 19 16:15 dataset/medical_twiteer_embeddings.npz
-rw------- 1 root root 2934926103 Jan 19 16:17 dataset/znany_lekarz_2k_embeddings.npz


In [6]:
!pip install -r requirements.txt

In [12]:
from model.dataset import load_embeddings
def load(path='dataset/medical_twiteer_embeddings.npz'):
    X, y, label_mappings = load_embeddings(path)
    print((X.shape, y.shape))
    print(label_mappings)
    return X, y, label_mappings

# Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, SimpleRNN
from tensorflow.keras.layers import LSTMCell, SimpleRNNCell, RNN, Bidirectional

#%%

from sklearn.metrics import f1_score
import numpy as np

def f1(y_true, y_pred):
    re = tf.argmax(y_pred, axis=1)
    re = tf.reshape(re, [-1,1])
    score = f1_score(y_true=y_true.numpy(), y_pred=re, average="macro")
    return score

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

#%%

n_categories = len(label_mappings)
print(f"n_categories={n_categories}")

#%%

model = Sequential()
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0)))
model.add(Dense(n_categories, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', f1]
              , run_eagerly=True #for tensor.numpy() in f1 score metric
              )

n_categories=3


In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=64, epochs=50, callbacks=[early_stopping])

Epoch 1/50
21/21 [==============================] - 3s 124ms/step - loss: 1.0042 - accuracy: 0.5161 - f1: 0.3441 - val_loss: 0.7628 - val_accuracy: 0.7002 - val_f1: 0.4948
Epoch 2/50
21/21 [==============================] - 2s 96ms/step - loss: 0.6735 - accuracy: 0.7370 - f1: 0.6305 - val_loss: 0.7911 - val_accuracy: 0.6708 - val_f1: 0.6003
Epoch 3/50
21/21 [==============================] - 2s 97ms/step - loss: 0.5303 - accuracy: 0.7839 - f1: 0.7132 - val_loss: 0.6926 - val_accuracy: 0.7187 - val_f1: 0.6651
Epoch 4/50
21/21 [==============================] - 2s 98ms/step - loss: 0.3904 - accuracy: 0.8601 - f1: 0.8175 - val_loss: 0.7224 - val_accuracy: 0.7141 - val_f1: 0.6632
Epoch 5/50
21/21 [==============================] - 2s 98ms/step - loss: 0.3255 - accuracy: 0.8851 - f1: 0.8583 - val_loss: 0.6939 - val_accuracy: 0.7496 - val_f1: 0.6720
Epoch 6/50
21/21 [==============================] - 2s 100ms/step - loss: 0.2313 - accuracy: 0.9213 - f1: 0.9111 - val_loss: 0.7215 - val_accura

In [ ]:
model.evaluate(X_test, y_test)

21/21 [==============================] - 1s 43ms/step - loss: 1.3091 - accuracy: 0.7125 - f1: 0.6310


[1.3090773820877075, 0.7125193476676941, 0.6310036778450012]

In [ ]:
y_test_pred = np.argmax(model.predict(X_test), axis=-1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred, target_names=list(label_mappings)))

                   precision    recall  f1-score   support

         Medyczne       0.61      0.74      0.67       219
Medyczne-diagnoza       0.46      0.48      0.47        67
      Niemedyczny       0.86      0.74      0.79       361

         accuracy                           0.71       647
        macro avg       0.64      0.65      0.64       647
     weighted avg       0.73      0.71      0.72       647



In [ ]:
type(label_mappings)

numpy.ndarray

In [ ]:
label_mappings

array(['Medyczne', 'Medyczne-diagnoza', 'Niemedyczny'], dtype='<U17')

In [ ]:
type(y_test_pred[0][0])

numpy.float32

In [ ]:
y_test_pred[0]

array([7.6998907e-01, 2.2961150e-01, 3.9949152e-04], dtype=float32)

In [ ]:
y_test[:10]

array([0, 2, 2, 2, 2, 2, 0, 0, 1, 2])

In [ ]:
y_train_2, y_test_2 = np.where(y_train == 2, 0, y_train), np.where(y_test == 2, 0, y_test)

In [ ]:
label_mappings_2 = ['Niemedyczny', 'Medyczne-diagnoza']

In [ ]:
n_categories = len(label_mappings_2)
print(f"n_categories={n_categories}")

#%%

model = Sequential()
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0)))
model.add(Dense(n_categories, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', f1]
              , run_eagerly=True #for tensor.numpy() in f1 score metric
              )

n_categories=2


In [ ]:
history = model.fit(X_train, y_train_2, validation_data=(X_test, y_test_2), batch_size=64, epochs=50, callbacks=[early_stopping])

Epoch 1/50
21/21 [==============================] - 3s 137ms/step - loss: 0.5199 - accuracy: 0.7435 - f1: 0.4038 - val_loss: 0.3312 - val_accuracy: 0.8980 - val_f1: 0.4976
Epoch 2/50
21/21 [==============================] - 2s 100ms/step - loss: 0.2983 - accuracy: 0.8867 - f1: 0.5306 - val_loss: 0.2662 - val_accuracy: 0.9026 - val_f1: 0.5914
Epoch 3/50
21/21 [==============================] - 2s 98ms/step - loss: 0.2311 - accuracy: 0.9148 - f1: 0.7299 - val_loss: 0.2517 - val_accuracy: 0.8980 - val_f1: 0.6743
Epoch 4/50
21/21 [==============================] - 2s 100ms/step - loss: 0.1613 - accuracy: 0.9431 - f1: 0.8201 - val_loss: 0.2576 - val_accuracy: 0.9042 - val_f1: 0.7059
Epoch 5/50
21/21 [==============================] - 2s 97ms/step - loss: 0.1323 - accuracy: 0.9492 - f1: 0.8498 - val_loss: 0.2649 - val_accuracy: 0.9104 - val_f1: 0.7161
Epoch 6/50
21/21 [==============================] - 2s 98ms/step - loss: 0.0940 - accuracy: 0.9683 - f1: 0.9178 - val_loss: 0.2866 - val_accur

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_7 (Bidirection (None, 128)               426496    
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 258       
Total params: 426,754
Trainable params: 426,754
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_test_pred = np.argmax(model.predict(X_test), axis=-1)
from sklearn.metrics import classification_report
print(classification_report(y_test_2, y_test_pred, target_names=list(label_mappings_2)))

                   precision    recall  f1-score   support

      Niemedyczny       0.94      0.97      0.95       580
Medyczne-diagnoza       0.61      0.42      0.50        67

         accuracy                           0.91       647
        macro avg       0.77      0.69      0.72       647
     weighted avg       0.90      0.91      0.90       647



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp dataset/medical_twiteer_embeddings.npz /content/drive/MyDrive/colab-data/sma-medical/

# Model builder

In [9]:
X, y, label_mappings = load()


((1958, 166, 768), (1958,))
['Medyczne' 'Medyczne-diagnoza' 'Niemedyczny']


In [8]:
from model.mkmodel import f1, ModelSet

In [9]:
ms = ModelSet(X, y, label_mappings)

In [ ]:
# model = ms.build_model()
# model.summary()
# ms.train_model(model)
# ms.print_report(model)

# Run experiments

In [10]:
from tensorflow.keras.layers import LSTM, GRU

In [ ]:
params = []
params.extend([{'layers': [l1], 'units': units} for units in [50, 100] for l1 in [LSTM, GRU]])
params.extend([{'layers': [l1, l2], 'units': 50} for l1 in [LSTM, GRU] for l2 in [LSTM, GRU]])
params.extend([{'layers': [l1, l2], 'units': 100} for l1 in [LSTM, GRU] for l2 in [LSTM, GRU]])
ms.build_and_train(params)
ms.collect_results()

Model: "LSTM 50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 100)               327600    
_________________________________________________________________
dense (Dense)                (None, 3)                 303       
Total params: 327,903
Trainable params: 327,903
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
25/25 [==============================] - 7s 100ms/step - loss: 0.9993 - accuracy: 0.5049 - f1: 0.2903 - val_loss: 0.7649 - val_accuracy: 0.6939 - val_f1: 0.5954
Epoch 2/50
25/25 [==============================] - 2s 88ms/step - loss: 0.6473 - accuracy: 0.7494 - f1: 0.6517 - val_loss: 0.6481 - val_accuracy: 0.7577 - val_f1: 0.7352
Epoch 3/50
25/25 [==============================] - 2s 88ms/step - loss: 0.5109 - accuracy: 0.7886 - f1: 0.7495 - val_loss: 0.6024 - val_accuracy: 0.7704 - val_f1: 0.7195
E

,name,accuracy,f1score
0,LSTM 50,0.808673,0.760417
1,GRU 50,0.798469,0.743465
2,LSTM 100,0.793367,0.745906
3,GRU 100,0.788265,0.717443
4,LSTM LSTM 50,0.793367,0.730475
5,LSTM GRU 50,0.775510,0.722367
6,GRU LSTM 50,0.790816,0.740866
7,GRU GRU 50,0.783163,0.734180
8,LSTM LSTM 100,0.762755,0.716303
9,LSTM GRU 100,0.798469,0.750575


# Results

In [ ]:
# import numpy as np
# np.savez_compressed('model/models.npz', models=ms.models)

In [ ]:
ms.collect_results()

13/13 [==============================] - 1s 43ms/step - loss: 0.9091 - accuracy: 0.8087 - f1: 0.7604
LSTM 50 [0.909082293510437, 0.8086734414100647, 0.760416567325592]
13/13 [==============================] - 1s 42ms/step - loss: 0.5957 - accuracy: 0.7985 - f1: 0.7435
GRU 50 [0.5956629514694214, 0.7984693646430969, 0.7434651255607605]
13/13 [==============================] - 1s 43ms/step - loss: 0.5637 - accuracy: 0.7934 - f1: 0.7459
LSTM 100 [0.5637373328208923, 0.793367326259613, 0.7459057569503784]
13/13 [==============================] - 1s 45ms/step - loss: 0.6427 - accuracy: 0.7883 - f1: 0.7174
GRU 100 [0.6426628828048706, 0.7882652878761292, 0.7174428701400757]
13/13 [==============================] - 1s 55ms/step - loss: 1.0510 - accuracy: 0.7934 - f1: 0.7305
LSTM LSTM 50 [1.050973892211914, 0.793367326259613, 0.7304754853248596]
13/13 [==============================] - 1s 56ms/step - loss: 1.2084 - accuracy: 0.7755 - f1: 0.7224
LSTM GRU 50 [1.2083746194839478, 0.77551019191741

,name,accuracy,f1score
0,LSTM 50,0.808673,0.760417
1,GRU 50,0.798469,0.743465
2,LSTM 100,0.793367,0.745906
3,GRU 100,0.788265,0.717443
4,LSTM LSTM 50,0.793367,0.730475
5,LSTM GRU 50,0.775510,0.722367
6,GRU LSTM 50,0.790816,0.740866
7,GRU GRU 50,0.783163,0.734180
8,LSTM LSTM 100,0.762755,0.716303
9,LSTM GRU 100,0.798469,0.750575


,name,accuracy,f1score
0,LSTM GRU 64,0.737245,0.679625
1,LSTM 50,0.767857,0.694516
2,GRU 50,0.757653,0.702924
3,LSTM 100,0.783163,0.698897
4,GRU 100,0.757653,0.712767
5,LSTM LSTM 50,0.747449,0.649755
6,LSTM GRU 50,0.750000,0.675231
7,GRU LSTM 50,0.801020,0.692491
8,GRU GRU 50,0.714286,0.661397
9,LSTM LSTM 100,0.795918,0.715428


In [ ]:
params=[]
params.extend([{'layers': [GRU, GRU], 'units': units} for units in [150, 200]])
ms.build_and_train(params)

Model: "GRU GRU 150"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_20 (Bidirectio (None, 166, 300)          828000    
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 300)               406800    
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 903       
Total params: 1,235,703
Trainable params: 1,235,703
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
25/25 [==============================] - 3s 119ms/step - loss: 1.0275 - accuracy: 0.5680 - f1: 0.3913 - val_loss: 0.7186 - val_accuracy: 0.6760 - val_f1: 0.5827
Epoch 2/50
25/25 [==============================] - 3s 115ms/step - loss: 0.5594 - accuracy: 0.7715 - f1: 0.6901 - val_loss: 0.5991 - val_accuracy: 0.7500 - val_f1: 0.7133
Epoch 3/50
25/25 [=============

In [ ]:
params=[]
params.extend([{'layers': [GRU, GRU, GRU], 'units': 100}])
ms.build_and_train(params)

Model: "GRU GRU GRU 100"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_24 (Bidirectio (None, 166, 200)          522000    
_________________________________________________________________
bidirectional_25 (Bidirectio (None, 166, 200)          181200    
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 200)               181200    
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 603       
Total params: 885,003
Trainable params: 885,003
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
25/25 [==============================] - 3s 138ms/step - loss: 1.0944 - accuracy: 0.4836 - f1: 0.2964 - val_loss: 0.6621 - val_accuracy: 0.7143 - val_f1: 0.5623
Epoch 2/50
25/25 [==============================] - 3s 127ms/step - los

In [ ]:
ms.collect_results()

13/13 [==============================] - 1s 43ms/step - loss: 0.9091 - accuracy: 0.8087 - f1: 0.7604
LSTM 50 [0.909082293510437, 0.8086734414100647, 0.760416567325592]
13/13 [==============================] - 1s 44ms/step - loss: 0.5957 - accuracy: 0.7985 - f1: 0.7435
GRU 50 [0.5956629514694214, 0.7984693646430969, 0.7434651255607605]
13/13 [==============================] - 1s 40ms/step - loss: 0.5637 - accuracy: 0.7934 - f1: 0.7459
LSTM 100 [0.5637373328208923, 0.793367326259613, 0.7459057569503784]
13/13 [==============================] - 1s 42ms/step - loss: 0.6427 - accuracy: 0.7883 - f1: 0.7174
GRU 100 [0.6426628828048706, 0.7882652878761292, 0.7174428701400757]
13/13 [==============================] - 1s 55ms/step - loss: 1.0510 - accuracy: 0.7934 - f1: 0.7305
LSTM LSTM 50 [1.050973892211914, 0.793367326259613, 0.7304754853248596]
13/13 [==============================] - 1s 53ms/step - loss: 1.2084 - accuracy: 0.7755 - f1: 0.7224
LSTM GRU 50 [1.2083746194839478, 0.77551019191741

,name,accuracy,f1score
0,LSTM 50,0.808673,0.760417
1,GRU 50,0.798469,0.743465
2,LSTM 100,0.793367,0.745906
3,GRU 100,0.788265,0.717443
4,LSTM LSTM 50,0.793367,0.730475
5,LSTM GRU 50,0.775510,0.722367
6,GRU LSTM 50,0.790816,0.740866
7,GRU GRU 50,0.783163,0.734180
8,LSTM LSTM 100,0.762755,0.716303
9,LSTM GRU 100,0.798469,0.750575


In [ ]:
!cp -r checkpoints/ /content/drive/MyDrive/colab-data/sma-medical/

# Run experiments on dr_herbert embeddings

In [13]:
!nvidia-smi

Tue Jan 19 17:10:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
X, y, label_mappings = load("dataset/dr_herbert_medical_twitter_embeddings.npz")

((2003, 141, 768), (2003,))
['Medyczne' 'Medyczne-diagnoza' 'Niejednoznaczny' 'Niemedyczny'
 'Remis-sporny']


In [50]:
import numpy as np

idxs = (y < 2) + (y == 3)
X = X[idxs]
y = y[idxs]
y[y == 3] = 2

label_mappings = list(np.concatenate((label_mappings[0:2], label_mappings[3:4])))
print(X.shape, y.shape)
print(label_mappings)

(1958, 141, 768) (1958,)
['Medyczne', 'Medyczne-diagnoza', 'Niemedyczny']


In [51]:
from model.mkmodel import f1, ModelSet
ms = ModelSet(X, y, label_mappings)

In [52]:
from tensorflow.keras.layers import LSTM, GRU

In [53]:
params = []
params.extend([{'layers': [l1], 'units': units} for units in [50, 100] for l1 in [LSTM, GRU]])
params.extend([{'layers': [l1, l2], 'units': 50} for l1 in [LSTM, GRU] for l2 in [LSTM, GRU]])
params.extend([{'layers': [l1, l2], 'units': 100} for l1 in [LSTM, GRU] for l2 in [LSTM, GRU]])
ms.build_and_train(params)
ms.collect_results()

Model: "LSTM-50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 100)               327600    
_________________________________________________________________
dense (Dense)                (None, 3)                 303       
Total params: 327,903
Trainable params: 327,903
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
49/49 [==============================] - 13s 70ms/step - loss: 0.9145 - accuracy: 0.5893 - f1: 0.3699 - val_loss: 0.6805 - val_accuracy: 0.7194 - val_f1: 0.5930
Epoch 2/50
49/49 [==============================] - 3s 68ms/step - loss: 0.6300 - accuracy: 0.7409 - f1: 0.6033 - val_loss: 0.6157 - val_accuracy: 0.7474 - val_f1: 0.6481
Epoch 3/50
49/49 [==============================] - 3s 68ms/step - loss: 0.5299 - accuracy: 0.7941 - f1: 0.7100 - val_loss: 0.6378 - val_accuracy: 0.7296 - val_f1: 0.6234
E

,name,accuracy,f1score
0,LSTM-50,0.757653,0.701750
1,GRU-50,0.739796,0.700692
2,LSTM-100,0.755102,0.686188
3,GRU-100,0.744898,0.688530
4,LSTM-LSTM-50,0.752551,0.714752
5,LSTM-GRU-50,0.762755,0.722502
6,GRU-LSTM-50,0.765306,0.727000
7,GRU-GRU-50,0.711735,0.691454
8,LSTM-LSTM-100,0.767857,0.720421
9,LSTM-GRU-100,0.732143,0.697925


In [54]:
!nvidia-smi

Tue Jan 19 17:53:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |   1135MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Run experiments on proper dr_herbert embeddings

In [10]:
X, y, label_mappings = load("dataset/dr_herbert_medical_twitter_embeddings_166.npz")

((1958, 166, 768), (1958,))
['Medyczne' 'Medyczne-diagnoza' 'Niemedyczny']


In [59]:
from model.mkmodel import f1, ModelSet
ms = ModelSet(X, y, label_mappings)

In [60]:
from tensorflow.keras.layers import LSTM, GRU

In [61]:
params = []
params.extend([{'layers': [l1], 'units': units} for units in [50, 100] for l1 in [LSTM, GRU]])
params.extend([{'layers': [l1, l2], 'units': 50} for l1 in [LSTM, GRU] for l2 in [LSTM, GRU]])
params.extend([{'layers': [l1, l2], 'units': 100} for l1 in [LSTM, GRU] for l2 in [LSTM, GRU]])
ms.build_and_train(params)
ms.collect_results()

Model: "LSTM-50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_20 (Bidirectio (None, 100)               327600    
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 303       
Total params: 327,903
Trainable params: 327,903
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
49/49 [==============================] - 4s 74ms/step - loss: 0.8544 - accuracy: 0.6018 - f1: 0.4142 - val_loss: 0.6488 - val_accuracy: 0.7194 - val_f1: 0.6603
Epoch 2/50
49/49 [==============================] - 4s 72ms/step - loss: 0.6049 - accuracy: 0.7401 - f1: 0.6591 - val_loss: 0.5949 - val_accuracy: 0.7500 - val_f1: 0.6838
Epoch 3/50
49/49 [==============================] - 3s 71ms/step - loss: 0.5230 - accuracy: 0.7904 - f1: 0.7057 - val_loss: 0.6451 - val_accuracy: 0.7526 - val_f1: 0.6281
Ep

,name,accuracy,f1score
0,LSTM-50,0.775510,0.717222
1,GRU-50,0.752551,0.694248
2,LSTM-100,0.770408,0.723445
3,GRU-100,0.780612,0.721327
4,LSTM-LSTM-50,0.772959,0.734049
5,LSTM-GRU-50,0.744898,0.719404
6,GRU-LSTM-50,0.795918,0.740938
7,GRU-GRU-50,0.765306,0.713380
8,LSTM-LSTM-100,0.762755,0.719519
9,LSTM-GRU-100,0.750000,0.720916


In [62]:
!nvidia-smi

Tue Jan 19 18:51:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    29W /  70W |   1647MiB / 15079MiB |      1%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------